In [11]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('ecommercedata-历史订单.xlsx')
df.head()

,InvoiceNo：发表号码,StockCode：产品代码,Description：产品描述,Quantity：每笔交易的每件产品数量,InvoiceDate:交易日期和时间,UnitPrice:单价（英镑）,CustomerID：顾客ID,Country：客户所在国家/地区
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [3]:
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# le = LabelEncoder()
# df['Country：客户所在国家/地区'] = le.fit_transform(df['Country：客户所在国家/地区'])
# df['InvoiceDate:交易日期和时间'] = le.fit_transform(df['InvoiceDate:交易日期和时间'])
# df.head(20)

In [4]:
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word

In [5]:
def gettext(x):
#     txt = open("piao.txt","r",errors='ignore').read()
#     txt = txt.lower()
    for ch in '!"#$&()*+,-./:;<=>?@[\\]^_{|}·~‘’\n @':
        txt = str(x).replace(ch,"")
    return txt

In [6]:
def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' ', tweet)
    tweet = re.sub(r'#GTCartoon:', ' ', tweet)
    return tweet

In [7]:
##判断是否为无效word
def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

In [8]:
def clean_text(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', ' ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', ' ', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    processed_tweet.append(tweet)
    words = tweet.lower().split()
    words = [w for w in words if w not in stop_words]
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            processed_tweet.append(word)
    return processed_tweet

In [9]:
stop_words = []
with open('常用英文停用词(NLP处理英文必备)stopwords.txt','r',encoding='utf-8')as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.strip().replace("'",""))

In [12]:
df['Description：产品描述'] = df['Description：产品描述'].apply(gettext)
df['Description：产品描述'] = df['Description：产品描述'].apply(preprocess_word)
df['Description：产品描述'] = df['Description：产品描述'].apply(handle_emojis)
df['Description：产品描述'] = df['Description：产品描述'].apply(clean_text)

In [13]:
df.head()

,InvoiceNo：发表号码,StockCode：产品代码,Description：产品描述,Quantity：每笔交易的每件产品数量,InvoiceDate:交易日期和时间,UnitPrice:单价（英镑）,CustomerID：顾客ID,Country：客户所在国家/地区
0,536365,85123A,"[white hanging heart tlight holder, white, han...",6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,"[white metal lantern, white, metal, lantern]",6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,"[cream cupid hearts coat hanger, cream, cupid,...",8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,"[knitted union flag hot water bottle, knitted,...",6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,"[red woolly hottie white heart, red, woolly, h...",6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [14]:
# coding=utf-8
import jieba
import jieba.analyse
import re
import time
from collections import Counter

#------------------------------------中文分词------------------------------------
cut_words = ""
all_words = ""
f = open('class-fenci.txt', 'w', encoding='utf-8')
for line in df['Description：产品描述']:
    cut_words = (" ".join(line))
    #计算关键词
    all_words = cut_words.split()
    c = Counter()
    for x in all_words:
        if len(x)>1 and x != '\r\n':
            c[x] += 1
    #Top50
    output = ""
    #print('\n词频统计结果：')
    for (k,v) in c.most_common():
        #print("%s:%d"%(k,v))
        output += k + " "
    
    f.write(output+"\n")
else:
    f.close()

In [15]:
#coding: utf-8
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#---------------------  第一步 读取数据(已分词)  ----------------------
corpus = []

# 读取预料 一行预料为一个文档
for line in open('class-fenci.txt', 'r',encoding='utf-8').readlines():
    corpus.append(line.strip())
        
#-----------------------  第二步 计算TF-IDF值  ----------------------- 
# 设置特征数
n_features = 2000

tf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stop_words,
                                max_df = 0.99,
                                min_df = 0.002) #去除文档内出现几率过大或过小的词汇

tf = tf_vectorizer.fit_transform(corpus)

print(tf.shape)

D:\rgzn\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['al'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(389168, 372)


In [16]:
from sklearn import cluster
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
if __name__ == "__main__":

    #########################################################################
    #                           第一步 计算TFIDF

    # 文档预料 空格连接
    corpus = []

    # 读取预料 一行预料为一个文档
    for line in open('class-fenci.txt', 'r',encoding='utf-8').readlines():
        corpus.append(line.strip())
    # 将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在i类文本下的词频
    vectorizer = CountVectorizer()

    # 该类会统计每个词语的tf-idf权值
    transformer = TfidfTransformer()

    # 第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵
    tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))
    # 获取词袋模型中的所有词语
    word = vectorizer.get_feature_names()

    # 将tf-idf矩阵抽取出来 元素w[i][j]表示j词在i类文本中的tf-idf权重
    weight = tfidf.toarray()
#     打印特征向量文本内容
    print('Features length: ' + str(len(word)))

    #
    ########################################################################
    #                               第二步 聚类Kmeans

    print('Start Kmeans:')
    from sklearn.cluster import KMeans

    clf = cluster.MiniBatchKMeans(n_clusters=3)
    print(clf)
    pre = clf.fit_predict(weight)
    print(pre)

Features length: 2047
Start Kmeans:
MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=3, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)
[0 0 0 ... 0 0 0]


In [17]:
counts = {}
for p in pre:
    counts[p] = counts.get(p,0)+1
counts

{0: 298282, 1: 55431, 2: 35455}

In [18]:
result = pd.concat((df,pd.DataFrame(pre)),axis=1)
result.rename({0:u'business value:商品商业价值'},axis=1,inplace=True)
result

,InvoiceNo：发表号码,StockCode：产品代码,Description：产品描述,Quantity：每笔交易的每件产品数量,InvoiceDate:交易日期和时间,UnitPrice:单价（英镑）,CustomerID：顾客ID,Country：客户所在国家/地区,business value:商品商业价值
0,536365,85123A,"[white hanging heart tlight holder, white, han...",6,2010-12-01 08:26:00,2.55,17850,United Kingdom,0
1,536365,71053,"[white metal lantern, white, metal, lantern]",6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0
2,536365,84406B,"[cream cupid hearts coat hanger, cream, cupid,...",8,2010-12-01 08:26:00,2.75,17850,United Kingdom,0
3,536365,84029G,"[knitted union flag hot water bottle, knitted,...",6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0
4,536365,84029E,"[red woolly hottie white heart, red, woolly, h...",6,2010-12-01 08:26:00,3.39,17850,United Kingdom,0
...,...,...,...,...,...,...,...,...,...
389163,C579886,22197,"[popcorn holder, popcorn, holder]",-1,2011-11-30 17:39:00,0.85,15676,United Kingdom,0
389164,C579886,23146,"[triple hook antique ivory rose, triple, hook,...",-1,2011-11-30 17:39:00,3.29,15676,United Kingdom,0
389165,C579887,84946,"[antique silver tlight glass, antique, silver,...",-1,2011-11-30 17:42:00,1.25,16717,United Kingdom,0
389166,C579887,85048,"[15cm christmas glass ball 20 lights, christma...",-1,2011-11-30 17:42:00,7.95,16717,United Kingdom,0


In [23]:
result.to_excel('new-ecommercedata-历史订单.xlsx')